In [1]:
import warnings
import pandas as pd
import collections
import numpy as np

from sklearn.metrics import confusion_matrix, classification_report, precision_score
from sklearn.decomposition import PCA

In [2]:
def QDA(Xapp, zapp, mprior, df_exp, Sprior, df_cov):
    n, p, K =  Xapp.shape[0], Xapp.shape[1], zapp.shape[1]
    pi = np.zeros(K)
    mu = np.zeros((K, p))
    B = np.zeros((n, p, p))
    Sig = np.zeros((p, p, K))
    
    pi = np.mean(zapp, axis=0)
    for j in range(K):
        mu[j, :] = (sum(zapp[i, j] * Xapp[i, :] for i in range(n)) 
                    + df_exp * mprior[j, :]) / (sum(zapp[:, j]) + df_exp)
        
        for i in range(n):
            temp = (Xapp[i, :] - mprior[j, :]).reshape((p, -1))
            B[i, :, :] = temp @ temp.T
        
        Sig[:, :, j] = (sum((zapp[i, j] * B[i, :, :]) for i in range(n)) + 
                        df_exp * (mu[j, :] - mprior[j, :]) @ (mu[j, :] - mprior[j, :]).T + 
                        Sprior[:, :, j]) / (sum(zapp[:, j]) + df_cov + p + 2)
        
    return pi, mu, Sig

In [3]:
def evaluation(Xtst, pi, mu, Sig):
    n, p, K = Xtst.shape[0], Xtst.shape[1], len(pi)
    prob = np.zeros((n, K))
    pred = np.zeros(n)
    
    f = lambda x, mu, Sig: (2 * np.pi)**(-n/2) * np.linalg.det(Sig)**(-1/2) * np.exp((-1/2) * (x - mu).T
                                                                                     @ np.linalg.pinv(Sig)
                                                                                     @ (x - mu))
   
    for j in range(K):
        for i in range(n):
            total = sum(pi[k] * f(Xtst[i, :], mu[k, :], Sig[:, :, k]) for k in range(K))
            prob[i, j] = pi[j] * f(Xtst[i, :], mu[j, :], Sig[:, :, j]) / total
    
    pred = np.argmax(prob, axis=1)
    
    return prob, pred

In [4]:
opt = pd.read_csv("dataset/optdigits.csv", header=0)
assert(opt.shape == (5620, 65))
opt = opt.to_numpy()

opt_y = opt[:, -1]
opt_X = opt[:, :-1]
pca = PCA(n_components=10)
opt_X = pca.fit_transform(opt_X)
opt = np.hstack((opt_X, opt_y.reshape((-1,1))))

In [5]:
n_opt, p_opt = opt.shape[0], opt.shape[1] - 1
K_opt = len(collections.Counter(list(opt[:, -1])))
cut_off = 5000

Xapp_opt = opt[0:cut_off, :-1]
Xtst_opt = opt[cut_off:, :-1]

Zapp_opt = np.zeros((cut_off, K_opt))
Ztst_opt = np.zeros((n_opt - cut_off, K_opt))

for i in range(cut_off):
    Zapp_opt[i, int(opt[i, -1])] = 1
for i in range(cut_off, n_opt):
    Ztst_opt[i - cut_off, int(opt[i, -1])] = 1

In [6]:
mprior_opt = np.zeros((K_opt, p_opt))
Sprior_opt = np.zeros((p_opt, p_opt, K_opt))
for j in range(K_opt):
    temp = opt[opt[:, -1] == j, :-1]
    mprior_opt[j, :] = temp.mean(axis = 0)
    Sprior_opt[:, :, j] = np.cov(temp, rowvar=False)

df_exp_opt = 10
df_cov_opt = p_opt + 300

In [7]:
pi_opt, mu_opt, Sig_opt = QDA(Xapp_opt, Zapp_opt, mprior_opt, df_exp_opt, Sprior_opt, df_cov_opt)
prob_opt, pred_opt = evaluation(Xtst_opt, pi_opt, mu_opt, Sig_opt)

In [8]:
print(confusion_matrix(np.argmax(Ztst_opt, axis=1), pred_opt))
print('\n\n')
print(classification_report(np.argmax(Ztst_opt, axis=1), pred_opt))
print('\n\n')
print(precision_score(np.argmax(Ztst_opt, axis=1), pred_opt, average='weighted'))

[[61  0  0  0  1  0  0  0  0  0]
 [ 0 58  0  0  0  0  0  0  0  5]
 [ 0  0 61  0  0  0  0  0  0  0]
 [ 0  0  0 55  0  2  0  2  4  0]
 [ 0  0  0  0 61  0  0  1  1  0]
 [ 0  0  0  0  0 62  0  0  1  0]
 [ 0  0  0  0  0  0 62  0  1  0]
 [ 0  0  0  0  0  0  0 62  0  0]
 [ 0  4  0  1  0  1  0  1 45  6]
 [ 0  0  0  1  0  2  0  0  0 59]]



              precision    recall  f1-score   support

           0       1.00      0.98      0.99        62
           1       0.94      0.92      0.93        63
           2       1.00      1.00      1.00        61
           3       0.96      0.87      0.92        63
           4       0.98      0.97      0.98        63
           5       0.93      0.98      0.95        63
           6       1.00      0.98      0.99        63
           7       0.94      1.00      0.97        62
           8       0.87      0.78      0.82        58
           9       0.84      0.95      0.89        62

    accuracy                           0.95       620
   macro avg    

In [9]:
pag = pd.read_csv("dataset/page-blocks.csv")
assert(pag.shape == (5473, 11))

In [10]:
sat = pd.read_csv("dataset/satimage.csv")
assert(sat.shape == (6430, 37))

In [11]:
seg = pd.read_csv("dataset/segment.csv")
assert(seg.shape == (2310, 20))

In [12]:
# 0.9766310415063318 1;p+20
# 0.9758926103474012 10;p+20
# 0.9671482548426473 10;p+300 
# v reprensent the least informative -> v high -> cov lower -> more "confident" ????
# high df_cov -> Sig =~~ sampling covariance matrix (in this case Sprior)